# The STEPS FOR DATA ANALISYS AND ML PROBLEMS :
- 1 - load and plot (optional) data to understand the problem
- 2 - (optional, recommended) data normalization / standardization - great influence on models that estimate on distances - gradient descent much faster
- 3 - (optional) feature selection / dimensionality reduction
- 4 - algorithm comparison and choice - train with cross validation ----- NB : scoring!!
- 5 - validation curve / learning curve
- 6 - (optional) algorithm tuning
- 7 - make predictions - confusion matrix & classification report


NB : always feature selection AFTER feature scaling

ps -> no validation curve or learning curve in this file. Search in the other if needed

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

from sklearn.preprocessing import Normalizer
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, RFE

#classification
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

#regression
from sklearn.svm import LinearSVR
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge #least squares
from sklearn.linear_model import Lasso
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score

%matplotlib inline

In [ ]:
data = pd.read_csv('weather.csv', sep=';')

DATA VISUALIZATION

In [ ]:
data.shape

In [ ]:
data.columns

In [ ]:
data.describe()

In [ ]:
data.dtypes

In [ ]:
# check for nan values
data[data.isna().any(axis=1)]

In [ ]:
# check for all zeros columns
data.columns[(data == 0).all()]

In [ ]:
# dataset bilanciato 50:50
data.groupby('y').size().plot(kind='bar')
plt.show()

In [ ]:
# visualize one column distribution
data.groupby('age').size().plot()
plt.show()

In [ ]:
# counting rows for each unique value (feature)
data.groupby('Location').size()

In [ ]:
# plotting with multi indexing - groupby su più chiavi

toplot = data.groupby(['marital', 'y']).size().unstack()
toplot.plot()

plt.show()

In [ ]:
# NBB
# plotting tha same things as before, but a graph for each first key
toplot = data.groupby(['marital', 'y']).size()

for i in toplot.index.levels[0]:
    toplot.loc[i].plot(kind='bar', title=i)
    plt.show()

In [ ]:
# per capire se una classe influisce su quella di predizione ---- y è quella di predizione
toplot = dataset.groupby(['marital', 'y']).size()
total = dataset.groupby('y').size()
# automaticamente posso dividere per ogni indice
(toplot / total).unstack().plot(kind='bar')
plt.show()

In [ ]:
# plottare la media di una coppia di feature
toplot = dataset.groupby(['sex','G3']).size()# do .unstack() after, so we can divide by index
total = dataset.groupby('sex').size()

(toplot / total).unstack().plot(kind='bar')
plt.show()

In [ ]:
# plot multiple curve on same graph
padri.plot(label='padri')
madri.plot(label='madri')
plt.xlabel('education')
plt.legend()
plt.show()

In [ ]:
# per identificare una picco in un istogramma su una feature
data['satisfaction_level'].hist(bins=100) # guardo istogramma per vedere numero di istanze
data.groupby('satisfaction_level').size() # associo valore ad istanze
data[data['satisfaction_level'] <= 0.11] # seleziono istanze
# confronto con dati normali con describe()

In [ ]:
# media e mediana di una feature
data['satisfaction_level'].describe()

# usare describe per confrontare due dataset / colonne

In [ ]:
hum = data[['Month', 'Location', 'Humidity9am', 'Humidity3pm']]
# per ogni coppia citta mese calcolo le temperature minime e massime calcolate al mattino e al pome
hum_max = hum.groupby(['Location', 'Month']).max()
hum_min = hum.groupby(['Location', 'Month']).min()
# qui cerco la massima/minima tra le due ----> mi fornisce quella giornaliera
hum_max = hum_max.max(axis=1)
hum_min = hum_min.min(axis=1)

DATA TRANSFORMATION

In [ ]:
# how to create a copy of a dataset
datanew = dataset.copy()

In [ ]:
# drop rows with nan instances
data = data.dropna() # axis=1 drop columns
# fillna valuese
data = data.fillna(0)

In [ ]:
# drop columns
data = data.drop([coltoremove], axis=1)
# drop rows
dataset_n = dataset_n.drop(toremove.index)

In [ ]:
#come si aggiungono colonne in modo idiomatico --- NB - viene messa alla fine

# new attributo escursione termica
data['escursione'] = data['MaxTemp'] - data['MinTemp']

# in alternativa va bene anche 
tmp = pd.DataFrame(data['MaxTemp'] - data['MinTemp'], columns=['escursione'])

#metto la colonna all'inizio -- si potrebbe anche concatenare (data[:, -1:],data[:, :-1])
data = tmp.join(data)


In [ ]:
# rimpiazzare valori nominali con numeri
data = data.replace(['Yes', 'No'],[1,0])
# same ----> data.replace({'yes':1, 'no':0})

#rimpiazzo solo una colonna
cities = data['Location'].unique()
data['Location'] = data['Location'].replace(cities, np.arange(len(cities)))

In [ ]:
# scegliere solo un subset di colonne da rimpiazzare
data.columns[data.dtypes == 'object']

In [ ]:
# rimpiazzo tutte le colonne con valori numerici
for col in tomap.columns:
    uniquev = tomap[col].unique()
    tomap.loc[:, col] = tomap.loc[:, col].replace(uniquev, np.arange(len(uniquev))) # loc non è necessario ma non da warning

In [ ]:
# always check values after
data.dtypes
# if some values still not int
data['Location'].unique()

In [ ]:
# dummies features su TUTTO il dataset ------ NB : ignora i valori float!!!!!! BUONO!
# NON IMPORTA SE I VALORI SONO STRINGHE ------ NON SERVE CONVERTIRE LE FEATURE IN NUMERI
data_dummy = pd.get_dummies(data)

In [ ]:
# creare un nuovo dataframe con dummies features solo su determinate colonne

new_data = data_numeric.copy()

todummy = 'age_new' # colonne da dummizzare could be data.dtypes == 'object'
dummies = pd.get_dummies(data_numeric[todummy])

new_data = new_data.drop(todummy, axis=1)
new_data = new_data.join(dummies)

In [ ]:
# creare nuovi dataset
# new datasets from orginal dataset
cloudP = datini[datini['Cloud3pm'] < 0]
cloudT = datini[-(datini['Cloud3pm'] < 0)] # the sign - revert a boolean series

In [ ]:
# ricevere dati numeri da dataframe
data_numeric = data._get_numeric_data()

In [ ]:
# database basato solo su alcune colonne
data_color = data.loc[:, [0,3,9,14,15,17,20]]

In [ ]:
# eventuale renaming delle colonne
data_color = data_color.rename(columns={0:'p',
                                        3:'c1',
                                        9:'c2',
                                        14:'c3',
                                        15:'c4',
                                        17:'c5',
                                        20:'c6'})

# se voglio rinominare tutte le colonne in con una lista
data_color.columns = ['p', 'c1', 'c2',...]

DATASET BALANCE

In [ ]:
# balance a data with resampling
# bigger rules or intermidiate value
from sklearn.utils import resample

bigger = data_air[data_air['SIZE']==1] 
smaller = data_air[data_air['SIZE']==3]

# bilanciamo le classi 
smaller = resample(smaller, replace=True, n_samples=5000) # can be also n_samples=len(bigger)
bigger = resample(bigger, replace=True, n_samples=5000) # non cessary if previous n_samples = len(bigger)

data_bal = pd.concat([bigger, smaller])
data_bal.groupby('SIZE').size()

DATA PREPARATION

NB posso spezzare i dati in mquesto modo se non mi è stata fornita nessuna specifica

In [ ]:
# nel caso la colonna di classificazione non sia in fondo possiamo usare questa notazione
X = data.drop('Cloud3pm', axis=1)
y = data['Cloud3pm']

In [ ]:
# dividere il dataset in un N intervalli 
# ---> ritorna array contenente indice di appartenenza ad un bin
interval = pd.cut(data_numeric['age'], bins=3, labels=False) 

In [ ]:
# nel caso dobbiamo dividere per ogni classe di size
# NBBBB-----> automaticamente con train_test_split() prendendo i valori a random
# ----------> le proprietà dovrebbero rimane quasi rispettate (ma non sarà perfetto)

index1 = data_bal['SIZE'] == 1
index3 = data_bal['SIZE'] == 3
index5 = data_bal['SIZE'] == 5

X_1 = data_bal[index1].drop('SIZE', axis=1)
y_1 = data_bal.loc[index1, 'SIZE']

X_3 = data_bal[index3].drop('SIZE', axis=1)
y_3 = data_bal.loc[index3, 'SIZE']

X_5 = data_bal[index5].drop('SIZE', axis=1)
y_5 = data_bal.loc[index5, 'SIZE']

In [ ]:
# andiamo poi ad unire le classi
fraction = 0.25

X_1train, X_1test, y_1train, y_1test = train_test_split(X_1, y_1, test_size=fraction)
X_3train, X_3test, y_3train, y_3test = train_test_split(X_3, y_3, test_size=fraction)
X_5train, X_5test, y_5train, y_5test = train_test_split(X_5, y_5, test_size=fraction)

X_train = pd.concat([X_5train, X_3train, X_1train])
X_test = pd.concat([X_5test, X_3test, X_1test])

y_train = pd.concat([y_5train, y_3train, y_1train])
y_test = pd.concat([y_5test, y_3test, y_1test])

In [ ]:
fraction = 0.2

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=fraction)

In [ ]:
# convert y to ndarray -----> solo se fatto senza train_test_split()

y_train = np.ravel(y_train)
y = np.ravel(y)

Non vado a normalizzare le componenti che sono attributi di tipo nominale, NON AVREBBE SENSO

In [ ]:
norm = Normalizer()
#X_new_train = norm.fit_transform(X_train)
#X_new_test = norm.fit_transform(X_test)

In [ ]:
# ora procedo al dimensionality reduction con pca
pca = PCA(n_components=10)
#X_new_train = pca.fit_transform(X_new_train)
#X_new_test = pca.fit_transform(X_new_test)

In [ ]:
# oppure con univariate
uni_sel = SelectKBest(k=10)
#X_new = uni_sel.fit_transform(X_new, y)

In [ ]:
# oppure con recursive feature elimination
#rfe = RFE()

In [ ]:
# distribuzione delle feature
tmp = pd.DataFrame(X_train)
tmp.hist(figsize=(12,5), bins=50)
plt.show()

In [ ]:
# scatter matrix
# può essere molto lento, da evitare con grandi dataset e molte features
# pd.plotting.scatter_matrix(tmp)
# plt.show()

In [ ]:
# correlation matrix
plt.matshow(tmp.corr())
plt.show()

implement a model (eg. logistic regression)

In [ ]:
# testare accuracy su modello
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

model = LogisticRegression(solver='lbfgs', max_iter=1000) # parameter change for each algorithms look down
model.fit(X_train, y_train)

predicted = model.predict(X_test)

print('accuracy :')
print(accuracy_score(y_test, predicted))
print('confusion matrix :')
print(confusion_matrix(y_test, predicted))
print('classification report :')
print(classification_report(y_test, predicted))

Running cross validation test on algoritmhs - accuracy/f1 score

In [ ]:
# cross validation on single model
model = DecisionTreeClassifier()
num_folds = 5
scoring = 'accuracy'

results = cross_val_score(model, X, y, scoring=scoring, cv=num_folds)

results.mean()

In [ ]:
# algorithm choice

models = []


#CLASSIFICATION

#models.append(('lSVM', LinearSVC() ))
#models.append(('SVM', SVC(gamma='scale') )) # can be also 'auto'
#models.append(('p2SVM', SVC(gamma='scale', kernel='poly', degree=2) ))
#models.append(('p3SVM', SVC(gamma='scale', kernel='poly', degree=3) ))
#models.append(('p4SVM', SVC(gamma='scale', kernel='poly', degree=4) ))
#models.append(('AB', AdaBoostClassifier() ))
#models.append(('NB', GaussianNB() ))
#models.append(('MNB', MultinomialNB() ))
models.append(('LR', LogisticRegression(solver='lbfgs', max_iter=1500) ))
models.append(('DT', DecisionTreeClassifier() ))
#models.append(('RF', RandomForestClassifier(n_estimators=10) ))
#models.append(('NN', MLPClassifier(max_iter=5000) )) troppo lento
#models.append(('KNN', KNeighborsClassifier() ))

# REGRESSION
#models.append(('lSVR', LinearSVR() ))
#models.append(('SVR', SVR(gamma='scale') ))
#models.append(('p2SVM', SVR(gamma='scale', kernel='poly', degree=2) ))
#models.append(('p3SVM', SVR(gamma='scale', kernel='poly', degree=3) ))
#models.append(('p4SVM', SVR(gamma='scale', kernel='poly', degree=4) ))
#models.append(('AB', AdaBoostRegressor() ))
#models.append(('LR', LinearRegression() ))
#models.append(('DT', DecisionTreeRegressor() ))
#models.append(('RI', Ridge() ))
#models.append(('LA', Lasso() ))
#models.append(('RF', RandomForestRegressor(n_estimators=10) ))
#models.append(('KNN', KNeighborsRegressor() ))


In [ ]:
# personalized scorer - eg. measuring accuracy of rounded regression results WITH CROSS VALIDATION
from sklearn.metrics import make_scorer

def accuracy_reg(y, y_pred, **kwargs):
    y_pred = np.round(y_pred)
    errors = np.sum(np.abs(y - y_pred))
    correct = len(y) - errors
    return correct/len(y)

# personal scorer instance
accuracy_r = make_scorer(accuracy_reg)
scoring = accuracy_r

In [ ]:
# scoring
# classification
scoring = 'accuracy' # 'f1_micro'
# regression
#scoring ='neg_mean_squared_error' # 'neg_mean_absolute_error' 

# testing models
results = []
names = []
num_folds = 5

for name, model in models:
    
    t = time.process_time()
    # be careful, use (X, y) or (X_train, y_train) depending on how data were splitted
    cv_result = cross_val_score(model, X_train, y_train, scoring=scoring, cv=num_folds)
    t = time.process_time() - t
    results.append(cv_result)
    names.append(name)
    
    print('time required by', name,' : ', t)
    
    #the confusione matrix è la matrice dei true positive ecc...
    
    #Xtrain, Xtest, ytrain, ytest = train_test_split(X_new, y)
    #model.fit(Xtrain, ytrain)
    #predicted = model.predict(Xtest)
    #print(confusion_matrix(ytest, predicted))
print('results for ', scoring, ':')
frame = pd.DataFrame(results, index=names)
frame.mean(axis=1)

In [ ]:
fig = plt.figure()
fig.suptitle('Algorithm comparison accuracy')
ax = fig.add_subplot(111)
plt.boxplot(frame)
ax.set_xticklabels(frame.index)
plt.show()